# Prototype of reaction-diffusion on temporal networks

Install the packages

In [7]:
Pkg.update()
Pkg.add("Distributions")
Pkg.add("LightGraphs")
Pkg.add("SimJulia")
Pkg.checkout("SimJulia")
Pkg.checkout("ResumableFunctions")

INFO: Updating METADATA...
INFO: Updating ResumableFunctions master...
INFO: Updating SimJulia master...
INFO: Computing changes...
INFO: No packages to install, update or remove
INFO: Package Distributions is already installed
INFO: Package LightGraphs is already installed
INFO: Package SimJulia is already installed
INFO: Checking out SimJulia master...
INFO: Pulling SimJulia latest master...
INFO: No packages to install, update or remove
INFO: Checking out ResumableFunctions master...
INFO: Pulling ResumableFunctions latest master...
INFO: No packages to install, update or remove


Making packages available in notebook

In [8]:
using Distributions
using LightGraphs
using ResumableFunctions
using SimJulia

Define the temporal behaviour of the network

In [9]:
@resumable function edge_behaviour(sim::Simulation, initial_state::Bool, src::Int, dst::Int, network::DiGraph, link_up::Event)
    up_distribution = Exponential(1.0) # other distributions are available
    down_distribution = Exponential(1.0) # other distributions are available
    next_state = initial_state
    while true
        if next_state
            println(now(sim), ": Edge from ", src, " to ", dst, " up")
            add_edge!(network, src, dst)
            next_state = false
            @yield Timeout(sim, rand(up_distribution))
        else
            println(now(sim), ": Edge from ", src, " to ", dst, " down")
            rem_edge!(network, src, dst)
            next_state = true
            @yield Timeout(sim, rand(down_distribution))
        end
    end
end

edge_behaviour (generic function with 1 method)

Define the walker behaviour

In [10]:
@resumable function walker_behaviour(sim::Simulation, walker::Int, vertex::Int, network::DiGraph, link_up::Event)
    wait_distribution = Exponential(1.0) # other distributions are available
    while true
        println(now(sim), ": Walker ", walker, " arrived in vertex ", vertex)
        @yield Timeout(sim, rand(wait_distribution))
        connections = out_neighbors(network, vertex)
        println(now(sim), ": Walker ", walker, " in vertex ", vertex, " has connections to ", connections)
        isempty(connections) && continue # no connections available, so we wait another period
        #if isempty(connections)
        #    src = 0
        #    while src != vertex
        #        println(now(sim), ": Walker ", walker, " in vertex ", vertex, " is waiting for connections")
        #        src = @yield link_up
        #        println(src)
        #    end
        #end
        connection_distribution = DiscreteUniform(1, length(connections))
        vertex = connections[Int(rand(connection_distribution))]
    end
end

walker_behaviour (generic function with 1 method)

Define the initial simulation settings

In [11]:
function play(run_time::Float64, nr_edges::Int, nr_walkers::Int)
    sim = Simulation()
    network = DiGraph(nr_edges) # specify the number of vertices
    edge_distribution = Bernoulli(0.5) # modify parameter to change the density of the initial network
    walker_distribution = DiscreteUniform(1, nr_edges)
    link_up = Event(sim)
    for src in 1:nr_edges
        for dst in 1:nr_edges
            src == dst && continue # self loops are not allowed
            state = Bool(rand(edge_distribution))
            @coroutine edge_behaviour(sim, state, src, dst, network, link_up)
        end
    end
    for walker in 1:nr_walkers
        vertex = Int(rand(walker_distribution))
        @coroutine walker_behaviour(sim, walker, vertex, network, link_up)
    end
    run(sim, run_time)
end

play (generic function with 1 method)

In [12]:
play(5.0, 3, 2)

0.0: Edge from 1 to 2 down
0.0: Edge from 1 to 3 down
0.0: Edge from 2 to 1 up
0.0: Edge from 2 to 3 down
0.0: Edge from 3 to 1 down
0.0: Edge from 3 to 2 down
0.0: Walker 1 arrived in vertex 2
0.0: Walker 2 arrived in vertex 2
0.09216423116628682: Walker 2 in vertex 2 has connections to [1]
0.09216423116628682: Walker 2 arrived in vertex 1
0.2939988985961515: Walker 2 in vertex 1 has connections to Int64[]
0.2939988985961515: Walker 2 arrived in vertex 1
0.2950049418689372: Edge from 1 to 3 up
0.404766368232377: Edge from 2 to 1 down
0.4323719602756899: Walker 2 in vertex 1 has connections to [3]
0.4323719602756899: Walker 2 arrived in vertex 3
0.5047717419131094: Edge from 1 to 2 up
0.5838546018456701: Edge from 2 to 3 up
0.6814218292834022: Edge from 2 to 1 up
0.8463546899370692: Walker 1 in vertex 2 has connections to [1, 3]
0.8463546899370692: Walker 1 arrived in vertex 3
0.8809533045116792: Edge from 3 to 1 up
1.0655642567842754: Walker 2 in vertex 3 has connections to [1]
1.0655